In [ ]:
import nltk
import sklearn
import pandas as pd
import gensim
import FuzzyTM
import contractions
import os
import re
import numpy as np
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt

import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

# Preprocessing
- What documents do we include in the corpus?
- How do we create the labels for the target variable?
- Has the corpus from Kaggle been pre-processed in some way? Do we use other proprocessing steps as well? (use at least stop-word removal)

In [ ]:
df_stock = pd.read_csv("../data/raw/AAPL.csv")

df_stock['Label'] = 0
df_stock.loc[(df_stock['Close'] - df_stock['Open']) > 0, 'Label'] = 1
df_stock

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df_stock['Date'], open=df_stock['Open'],
                high=df_stock['High'],
                low=df_stock['Low'],
                close=df_stock['Close'])])

fig.update_layout(
    autosize=False,
    width=1500,
    height=800,)
fig.show()

In [ ]:
df_text = pd.read_csv("../data/raw/us_equities_news_dataset.csv")

In [ ]:
df_text

## Select relevant documents (corpus)

In [ ]:
print("There are {} documents of which the title contains the word Apple".format(str(df_text['title'].str.contains('[Aa]pple').sum())))
print("There are {} documents of which the title contains the word iPad".format(str(df_text['title'].str.contains('i[Pp]ad').sum())))
print("There are {} documents of which the title contains the word iPod".format(str(df_text['title'].str.contains('i[Pp]od').sum())))
print("There are {} documents of which the title contains the word iPhone".format(str(df_text['title'].str.contains('i[Pp]hone').sum())))
print("There are {} documents of which the title contains the word Steve Jobs".format(str(df_text['title'].str.contains('[Ss]teve [Jj]obs').sum())))
print("There are {} documents of which the title contains the word Tim Cook".format(str(df_text['title'].str.contains('[Tt]im [Cc]ook').sum())))
print("There are {} documents of which the title contains the word iOS".format(str(df_text['title'].str.contains('[Ii][Oo][Ss]').sum())))
print("There are {} documents of which the title contains the word MacOS".format(str(df_text['title'].str.contains('[Mm]ac[Oo][Ss]').sum())))
print("There are {} documents of which the title contains the word Macbook".format(str(df_text['title'].str.contains('[Mm]ac[Bb]ook').sum())))
print("There are {} documents of which the title contains the word Mac Pro".format(str(df_text['title'].str.contains('[Mm]ac [Pp]ro').sum())))

In [ ]:
doc_idx = [] # list of indexnumbers that include above titel RegEx
for i in ['[Aa]pple', 'i[Pp]ad', 'i[Pp]od', 'i[Pp]hone', '[Ss]teve [Jj]obs', '[Tt]im [Cc]ook']:
    idx_list = df_text[df_text['title'].str.contains(i)].index
    for idx in idx_list:
        if idx not in doc_idx:
            doc_idx.append(idx)
            
print(len(np.unique(doc_idx)))

In [ ]:
for idx in list(df_text.loc[df_text['ticker'] == 'AAPL'].index):
    doc_idx.append(idx)
    
print(len(np.unique(doc_idx)))

In [ ]:
doc_idx = np.unique(doc_idx)

In [ ]:
corpus = df_text.iloc[doc_idx].reset_index(drop=True)

## Add stock movement label & match dates
- Add stock movement label to each news article in corpus.
    - Additionaly, it sorts on the date
    - Additionaly, it matches the dates for the two data source because of the inner join
    - This means we only include days that have both news articles and (recorded) stock movements

In [ ]:
corpus = corpus.merge(df_stock.loc[:,['Date','Label']], on=None, left_on='release_date', right_on='Date', copy=False, sort=True).drop(['Date'], axis=1)
corpus


## Remove duplicates

In [ ]:
print(f"{corpus.shape[0]} original number of articles")
corpus.loc[corpus.astype(str).drop_duplicates().index]
print(f"{corpus.shape[0]} after dropping perfect duplicates")
corpus.drop_duplicates(subset=['content'], inplace=True)
print(f"{corpus.shape[0]} after dropping duplicates on content")
corpus.drop_duplicates(subset=['title','release_date'], inplace=True)
print(f"{corpus.shape[0]} after dropping duplicates on title and release_date only")

## Contractions removal

In [ ]:
for i in range(len(corpus)):
    corpus.iloc[[i],[4]] = contractions.fix(corpus.iloc[4,4])

## Tokenize text

In [ ]:
corpus['unigrams'] = corpus.apply(lambda row: nltk.word_tokenize(row['content']), axis=1)

## Remove article markup

In [ ]:
corpus.iloc[0,10]

In [ ]:
corpus.iloc[0,4]

In [ ]:
# Remove \n

## Removing stop words & lowercasing

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
for idx, document in corpus.iterrows():
    #print(len(document['unigrams']))
    no_stop_words= [token.lower () for token in document['unigrams'] if not token.lower () in stop_words]
    #print(len(document['unigrams']))
    corpus.at[idx, 'unigrams'] = no_stop_words
    #print("====" )

## Lemmatization

In [ ]:
# Lemmatization (takes about 16 seconds)
lemmatizer = WordNetLemmatizer()

for idx, document in corpus.iterrows():
    #print(len(document['unigrams']))
    lemmings= [lemmatizer.lemmatize(token) for token in document['unigrams']]
    #print(len(document['unigrams']))
    corpus.at[idx, 'unigrams'] = lemmings
    #print("====" )

## Punctuation removal

In [ ]:
for idx, document in corpus.iterrows():
    #print(len(document['unigrams']))
    no_punctuation = [token for token in document['unigrams'] if token.isalpha()]
    corpus.at[idx, 'unigrams'] = no_punctuation
    #print(len(document['unigrams']))
    #print("======" )


## Remove nonsensical words

In [ ]:
english_words = set(words.words())
for idx, document in corpus.iterrows():
    #print(len(document['unigrams']))
    no_nonsense = [token for token in document['unigrams'] if token in english_words]
    corpus.at[idx, 'unigrams'] = no_nonsense

## Top word count

In [ ]:
from collections import Counter
counted_doc = Counter(corpus['unigrams'][2])
counted_doc.most_common(10)

# Modeling
- Use BoW representation. But there are still many modeling decisions to be made. Motivate these decisions, pay attention to feature selection and the measures/metrics you use to represent the words/text.
- Which classification algorithm do we choose? Why?
- Which parameters do we use? Why?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

X = vectorizer.fit_transform(corpus['unigrams'][:])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

In [ ]:
df_bow_sklearn.columns[1000:10000]

In [ ]:
df_bow_sklearn.sum(axis=0).sort_values(ascending = True)[:int(len(df_bow_sklearn.columns)*0.3)]

In [ ]:
(df_bow_sklearn.sum(axis=0) >= 1).sum()

In [ ]:
(df_bow_sklearn.sum(axis=0) > 100).sum()

In [ ]:
df_bow_sklearn.loc[:,'dumb'].sum(axis=0)

In [ ]:
# Create the tf-idf representation using the bag-of-words matrix
tfidf_transform = sklearn.feature_extraction.text.TfidfTransformer(norm=None)
# X_tfidf = tfidf_transform.fit_transform(df_bow_sklearn)

In [ ]:
# X_tfidf

# Train & Evaluate
- What is the experimental design used for training and evaluating?
- How does the model perform? What metrics do we use and why?